In [2]:
import numpy as np
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from mcda.matrices import PerformanceTable
from mcda.scales import QuantitativeScale, PreferenceDirection
from mcda.outranking.electre import Electre3

from methods.electre_complete import electre_complete_pulp
from methods.electre_partial import electre_partial_pulp

In [3]:
labels = ["A1", "A2", "A3", "A4", "A5", "A6", "A7"]

weights = [8, 4, 7, 2, 5]

# Definicja alternatyw
alternatives = [
    "A1",
    "A2",
    "A3",
    "A4",
    "A5",
    "A6",
    "A7"
]

# Definicja skali dla każdego kryterium
scales = {
    0: QuantitativeScale(1000, 4000, preference_direction=PreferenceDirection.MIN),# Price 
    1: QuantitativeScale(32, 1024, preference_direction=PreferenceDirection.MAX),    # Memory Size
    2: QuantitativeScale(8, 120, preference_direction=PreferenceDirection.MAX),     # Camera 
    3: QuantitativeScale(0, 5, preference_direction=PreferenceDirection.MAX),      # Functionalities 
    4: QuantitativeScale(12, 32, preference_direction=PreferenceDirection.MAX),      # Battery life
}

dataset = PerformanceTable(
    [[2200, 256, 32, 2, 24], [3200, 128, 64, 2, 18], [4000, 128, 32, 3, 22], [2400, 64, 32, 2, 28], [1400, 256, 120, 4, 14], [1600, 128, 120, 3, 14], [3200, 512, 16, 2, 20]],
    alternatives=alternatives, scales=scales
)
# Wagi kryteriów
W = {
    0: weights[0],  # Price 
    1: weights[1],   # Memory Size
    2: weights[2],  # Camera 
    3: weights[3],  # Functionalities 
    4: weights[4]   # Battery life
}

# Parametry Electre III

I = {0: 300, 1: 64,  2: 12, 3: 2,      4: 2}
P = {0: 600, 1: 256, 2: 32, 3: 3,      4: 6}
V = {0: 900, 1: 512, 2: 64, 3: np.inf, 4: 12}

In [4]:
electre3 = Electre3(performance_table=dataset, 
                    criteria_weights=W, 
                    indifference_thresholds=I, 
                    preference_thresholds=P, 
                    veto_thresholds=V)

In [5]:
credibility = electre3.credibility().data.values
electre_matrix = credibility.copy()

In [9]:
print(electre_matrix.round(2))

[[1.   0.73 1.   0.9  0.   0.   0.85]
 [0.   1.   0.9  0.22 0.   0.   0.85]
 [0.   0.24 1.   0.   0.   0.   0.39]
 [0.9  0.73 1.   1.   0.   0.   0.85]
 [0.81 0.9  0.81 0.   1.   1.   0.65]
 [0.76 0.9  0.81 0.   0.95 1.   0.65]
 [0.   0.73 0.95 0.27 0.   0.   1.  ]]


In [12]:
r_ec, z_ec, obj_ec = electre_complete_pulp(electre_matrix)

print("ELECTRE COMPLETE\n")
print(r_ec)
print(obj_ec)

ELECTRE COMPLETE

[[0 1 1 0 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 1 0 1 1 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1]
 [0 1 1 0 0 0 0]]
20.206267806267803


In [13]:
r_ep, Pp_ep, Pm_ep, I_ep, R_ep, obj_ep = electre_partial_pulp(electre_matrix)

print("ELECTRE COMPLETE\n")
print(r_ep)
print(obj_ep)

ELECTRE COMPLETE

[[0 1 1 0 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 1 1 0 0 0 1]
 [1 1 1 0 0 1 1]
 [1 1 1 0 1 0 1]
 [0 1 1 0 0 0 0]]
16.1037037037037
